# Fluent Python
https://github.com/fluentpython/example-code

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated

## CHAPTER 4 Text versus Bytes

유니코드 무자열, 이진 시퀀스, 그리고 이 두란의 변환에 사용되는 인코딩 
### Byte Essentials

In [6]:
s = 'Cafe'
b = s.encode('utf8')
b, b.decode('utf8')

(b'Cafe', 'Cafe')

- bytes : 불변형
- bytearray : 가변형

In [11]:
cafe = bytes('café', encoding='utf_8')
print(cafe[0]) # int 형을 반환
cafe[:1] # bytes 형을 반환

99


b'c'

In [15]:
cafe_arr = bytearray(cafe)
print(type(cafe_arr[0]))
cafe_arr[-1:]

<class 'int'>


bytearray(b'\xa9')

str 만이 s[0]==s[:1] 슬라이스한 값과 특정 값이 동일한 형태를 반환한다. (예외적)

버퍼와 같은 객체로 부터 bytes 나 bytearray 객체를 생성하면 언제나 바이트를 직접 복사 

memeoryview 는 이진데이터 구조체 간의 메모리를 공유

`memoryview` : 내장 클래스, 공유 메모리 시퀀스형으로서 bytes를 복사하지 않고 매열의 슬라이스를 다룰 수 있게 해줌. NumPy 배열 구조체를 일반화한것. 데이터 구조체를 복사하지 않고 메모리를 공유할 수 있게 해줌. 언제나 동일한 메모리를 공유. 슬라이스하게 되면 새로운 memoryview 객체가 반환됨. 

`struct` : 패킹된 바이트를 당양한 필드로 구성된 튜플로 분석, 이와 반대로 튜플을 패킹된 바이트로 변환하는 함수 제공.

### Basic Encoders/Decoders

In [16]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


### Understanding Encode/Decode Problems

In [17]:
city = 'São Paulo'
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [18]:
city.encode('cp437', errors='replace')
city.encode('cp437', errors='xmlcharrefreplace') 
# xmlcharrefreplace 처리기는 인코딩 할수 없는 문자를 XML 객체로 치환한다.

b'S?o Paulo'

b'S&#227;o Paulo'

디코딩시 많은 레거시 8 비트 코덱은 무작위 비트 배열에 대해서 에러를 발생시키지 않는다. 

In [19]:
# These bytes are the characters for “Montréal” encoded as latin1 ; '\xe9' is the byte for “é”.
octets = b'Montr\xe9al'
octets.decode('utf_8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [20]:
octets.decode('utf_8', errors='replace')

'Montr�al'

인코딩 탐지하는 `Chardet`

인코딩된 텍스트의 이진 시퀀스는 인코딩에 대한 정보를 명시적으로 전달하지 않지만, UTF 포맷은 텍스트 앞에 바이트 순서를 표시(Byte Order Mark) 추가 할수 있다.

In [22]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

## Handling Text Files

샌드위치 방법

byte -> str 은 가장 빠르게 바꿔서 쓰고 <br>
str 을 이용해 비즈니스 로직을 돌리고 <br>
str -> byte 는 가장 느리게 다쓰고 바꿔라

In [23]:
open('cafe.txt', 'w', encoding='utf_8').write('café')
open('cafe.txt', 'r', encoding='utf_8').read() # 꼭 인코딩을 지정해줘야 한다. 

4

'café'

### Encoding Defaults: A Madhouse

In [24]:
import sys, locale
expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""
my_file = open('dummy', 'w')
for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


### Normalizing Unicode for Saner Comparisons

Normalization Form C (NFC) composes the code points to produce the shortest equiv‐alent string, while NFD decomposes, expanding composed characters into base char‐acters and separate combining characters.


In [25]:
from unicodedata import normalize
s1 = 'café' # composed "e" with acute accent
s2 = 'cafe\u0301' # decomposed "e" and acute accent

s1 == s2

False

In [26]:
normalize('NFC', s1) == normalize('NFC', s2)
normalize('NFD', s1) == normalize('NFD', s2)

True

True

In [27]:
"""
Utility functions for normalized Unicode string comparison.
    Using Normal Form C, case sensitive:
        >>> s1 = 'café'
        >>> s2 = 'cafe\u0301'
        >>> s1 == s2
        False
        >>> nfc_equal(s1, s2)
        True
        >>> nfc_equal('A', 'a')
        False
    Using Normal Form C with case folding:
        >>> s3 = 'Straße'
        >>> s4 = 'strasse'
        >>> s3 == s4
        120
        | Chapter 4: Text versus BytesFalse
        >>> nfc_equal(s3, s4)
        False
        >>> fold_equal(s3, s4)
        True
        >>> fold_equal(s1, s2)
        True
        >>> fold_equal('A', 'a')
        True
"""
from unicodedata import normalize
def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)
def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold())

"\nUtility functions for normalized Unicode string comparison.\n    Using Normal Form C, case sensitive:\n        >>> s1 = 'café'\n        >>> s2 = 'café'\n        >>> s1 == s2\n        False\n        >>> nfc_equal(s1, s2)\n        True\n        >>> nfc_equal('A', 'a')\n        False\n    Using Normal Form C with case folding:\n        >>> s3 = 'Straße'\n        >>> s4 = 'strasse'\n        >>> s3 == s4\n        120\n        | Chapter 4: Text versus BytesFalse\n        >>> nfc_equal(s3, s4)\n        False\n        >>> fold_equal(s3, s4)\n        True\n        >>> fold_equal(s1, s2)\n        True\n        >>> fold_equal('A', 'a')\n        True\n"

In [28]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2
nfc_equal(s1, s2)
nfc_equal('A', 'a')

False

True

False

In [29]:
s3 = 'Straße'
s4 = 'strasse'
s3 == s4
nfc_equal(s3, s4)
fold_equal(s3, s4)
fold_equal(s1, s2)
fold_equal('A', 'a')

False

False

True

True

True

### Dual-Mode str and bytes APIs

`re` 모듈과 `os` 모듈

str Versus bytes on os Functions



In [30]:
import os

os.listdir('.')

['.ipynb_checkpoints',
 'cafe.txt',
 'dummy',
 'Julie_CH4.Text versus Bytes.ipynb',
 'tmp.txt']

In [31]:
os.listdir(b'.')

[b'.ipynb_checkpoints',
 b'cafe.txt',
 b'dummy',
 b'Julie_CH4.Text versus Bytes.ipynb',
 b'tmp.txt']

- fsencode(파일명) : 파일명이 str 형이면 sys.getfilesystemencoding() 이 반환한 코덱명을 이용해서 '파일명'을 bytes 형으로 인코딩한다. '파일명' 이 byte 형이면 변환하지 않고 그대로 반환한다.
- fsdecode(파일명) : 파일명이 byte 형이면 sys.getfilesystemencoding() 이 반환한 코덱명을 이용해서 '파일명'을 str 형으로 디코딩한다. '파일명' 이 str 형이면 변환하지 않고 그대로 반환한다.

surrogateescape 코덱에러처리기